# Préparation des variables de ses

In [19]:
library(tidyverse)
library(factoextra)

# Charger les données
df_pilot1_2022 <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/qc2022/01_pilot1_2022.rds")

# Charger le fichier des variables
source("01-1_variables.R")

df_ses <- df_pilot1_2022 %>%
  select(all_of(variables_ses))

In [20]:
# Boucle pour afficher la table pour chaque variable
for (v in variables_ses) {
  if (v != "id") {
    cat("Table pour la variable:", v, "\n")
    print(table(df_ses[[v]]))
    cat("\n\n")
  }
}

Table pour la variable: male 

  0   1 
814 686 


Table pour la variable: female 

  0   1 
695 805 


Table pour la variable: age34m 

   0    1 
1147  353 


Table pour la variable: age3554 

  0   1 
958 542 


Table pour la variable: age55p 

  0   1 
895 605 


Table pour la variable: langEn 

   0    1 
1287  213 


Table pour la variable: langFr 

   0    1 
 249 1251 


Table pour la variable: ses_languageOther 

   0    1 
1464   36 


Table pour la variable: educBHS 

   0    1 
1110  390 


Table pour la variable: educCollege 

  0   1 
957 543 


Table pour la variable: educUniv 

  0   1 
933 567 


Table pour la variable: ses_income_None 

   0    1 
1494    6 


Table pour la variable: ses_income_i1to30 

   0    1 
1302  198 


Table pour la variable: ses_income_i31to60 

   0    1 
1159  341 


Table pour la variable: ses_income_i61to90 

   0    1 
1206  294 


Table pour la variable: ses_income_i91to110 

   0    1 
1321  179 


Table pour la variable: ses_income_i1

### Transformations

Les variables age, éducation et income devraient être des variables ordinales

In [21]:
# Transformation de age en variable ordinale
#-------------------------------------------

df_ses <- df_ses %>%

  mutate(
    age = if_else(
      age34m == 1,
      0,
      if_else(
        age3554 == 1,
        0.5,
        1
      )
    )
  ) %>%
  select(
    -age34m,
    -age3554,
    -age55p
  )

In [22]:
# Transformation de educ en variable ordinale
#--------------------------------------------

df_ses <- df_ses %>%

  mutate(
    educ = if_else(
      educBHS == 1,
      0,
      if_else(
        educCollege == 1,
        0.5,
        1
      )
    )
  ) %>%
  select(
    -educBHS,
    -educCollege,
    -educUniv
  )

In [23]:
# Transformation de income en variable ordinale
#----------------------------------------------

df_ses <- df_ses %>%

  mutate(
    ses_income_None = 0,
    ses_income_i1to30    = ses_income_i1to30/7,
    ses_income_i31to60   = (1*ses_income_i31to60)/7 + ses_income_i31to60/7,
    ses_income_i61to90   = (2*ses_income_i61to90)/7 + ses_income_i61to90/7,
    ses_income_i91to110  = (3*ses_income_i91to110)/7 + ses_income_i91to110/7,
    ses_income_i111to150 = (4*ses_income_i111to150)/7 + ses_income_i111to150/7,
    ses_income_i151to200 = (5*ses_income_i151to200)/7 + ses_income_i151to200/7,
    ses_income_i201toInf = (ses_income_i201toInf)
  ) %>%
  mutate (
    ses_income = ses_income_None +
      ses_income_i31to60 +
      ses_income_i61to90 +
      ses_income_i91to110 +
      ses_income_i111to150 +
      ses_income_i151to200 +
      ses_income_i201toInf
  ) %>%
  select(
    -ses_income_None,
    -ses_income_i201toInf,
    -ses_income_i31to60,
    -ses_income_i61to90,
    -ses_income_i91to110,
    -ses_income_i111to150,
    -ses_income_i151to200,
    -ses_income_i201toInf
  )

In [24]:
# Transformation de l'orientation sexuelle 
#----------------------------------------------

df_ses <- df_ses %>%

  mutate(ses_queer = if_else(ses_gai == 1 | ses_bisex == 1 | ses_sexOri_other == 1,
  1, 0)) %>%
    select(-ses_gai, -ses_bisex, -ses_sexOri_other)

In [25]:
variables_ses_clust <- c(
  "id",
  "male",
  "female",
  "age",
  "langEn",
  "langFr",
  "ses_languageOther",
  "educ",
  "ses_income",
  "immigrant",
  "ses_ethn_White",
  "ses_ethn_Black",
  "ses_ethn_Other",
  "ses_hetero",
  "ses_queer"
)

In [26]:
# Boucle pour afficher la table pour chaque variable
for (v in variables_ses_clust) {
  if (v != "id") {
    cat("Table pour la variable:", v, "\n")
    print(table(df_ses[[v]]))
    cat("\n\n")
  }
}

Table pour la variable: male 

  0   1 
814 686 


Table pour la variable: female 

  0   1 
695 805 


Table pour la variable: age 

  0 0.5   1 
353 542 605 


Table pour la variable: langEn 

   0    1 
1287  213 


Table pour la variable: langFr 

   0    1 
 249 1251 


Table pour la variable: ses_languageOther 

   0    1 
1464   36 


Table pour la variable: educ 

  0 0.5   1 
390 543 567 


Table pour la variable: ses_income 

                0 0.285714285714286 0.428571428571429 0.571428571428571 
              380               341               294               179 
0.714285714285714 0.857142857142857                 1 
              170                96                40 


Table pour la variable: immigrant 

   0    1 
1309  191 


Table pour la variable: ses_ethn_White 

   0    1 
 150 1350 


Table pour la variable: ses_ethn_Black 

   0    1 
1475   25 


Table pour la variable: ses_ethn_Other 

   0    1 
1471   29 


Table pour la variable: ses_hetero 

   0    1 

In [27]:
# Sauvegarder les données préparées
saveRDS(df_ses, file = "../../_PrivateFolder_datagotchi_federal_2025/data/clustering/qc2022/02_pilot1_2022_ses.rds")

---

# SUGGESTION : Activité de Clustering

Si souhaité, effectuer un exercice de clustering avec ce sous-groupe de variables pour mieux saisir les données et leur relation. 

Cet exercice est purement exploratoire et sert à voir si les données peuvent être utilisées pour faire du clustering.